## **Подобрать рецепты блюд по ингредиентам**

### **Цели, задачи, принципы**

На сайте [webspoon.ru](https://webspoon.ru/ingridients) можно выбрать от 1 до 4 ингредиентов из списка и найти рецепты с ними.

**Алгоритм действий пользователя**: 

* задать драйвер для selenium с необходимыми параметрами

* Подать в функцию find_recipes перечень из 1-4 ингредиентов (это может быть строка для одного ингредиента или лист/ кортеж/ множество/ pd.Series для 1 или нескольких ингредиентов) и драйвер для Selenium.\
Название ингредиента должно подаваться в функцию в том виде, как оно пишется на сайте, например, "Нут (турецкий горох)", а не просто "нут". При этом можно писать как заглавными, так и строчными буквами.

* Получить на выходе таблицу pd.DataFrame с названием рецепта, перечнем ингредиентов и их кол-вом и ссылкой на рецепт

**Алгоритм кода**:
Основной функцией, которую нужно использовать для получения таблицы с рецептами по заданным ингредиентам, является **find_recipes**.

Эта функция:
* По поданному списку ингредиентов проверяет, есть ли такие на сайте. Если хотя бы одного на сайте нет, возвращает None и печатает предупреждение вида "Ингредиента N не нашлось на сайте", а также возможные подходящие варианты их правильного написания (если таковые есть).\
Для этой части используются 3 дополнительные функции:\
    **get_ings** получает актуальный список ингредиентов с сайта,\
    **check_ings** проверяет, есть ли поданные пользователем аргументы на сайте,\
    **str_lower** используется для перевода всех названий ингредиентов в нижний регистр и обрезки пробелов снаружи, чтобы сделать поиск нечувствительным к регистру

* Если все поданные пользователем ингредиенты находятся на сайте, функция find_recipes кликает по ним, затем по кнопке - Подобрать рецепты, после чего, если не нашлось ни одного рецепта, возвращает None и печатает "На сайте не нашлось рецептов с выбранными ингредиентами, а если они нашлись, то забирает все рецепты с первой страницы (максимально их бывает около 50 на одной странице, это самые популярные рецепты с выбранными ингредиентами), создаёт с ними pd.DataFrame, возвращает неё и сообщение "Приятного аппетита!".

### **Драйвер для Selenium**

В моём случае драйвер открывается для Firefox и предполагается, что он лежит в рабочей папке.

Если нужно задать драйвер для другого браузера и с другими параметрами, перезапишите переменные profile и driver соответствующим образом.

In [1]:
from fake_useragent import UserAgent
from selenium import webdriver
    
useragent = UserAgent()
profile = webdriver.FirefoxProfile()
profile.set_preference('general.useragent.override', useragent.random)
driver = webdriver.Firefox(firefox_profile=profile)

### **Функция для запроса актуального перечня ингредиентов с сайта**

**Аргументы**:
* driver - драйвер для Selenium

**На выходе**:
* pandas data frame c колонками: Group (Группа ингредиентов), Ingredient (Ингредиент).

In [2]:
def get_ings(driver):
    from bs4 import BeautifulSoup
    from time import sleep
    import pandas as pd
    
    # задан ли драйвер для Selenium?
    try:
        driver.get('https://webspoon.ru/ingridients')
    except:
        print('Задайте драйвер для Selenium (так, чтобы новое окно браузера было открыто).')
        return
    sleep(10)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    ing_groups = soup.find_all('div', {'class': 'Ingredients-group'})
    
    all_ings = {
        'Group': [],
        'Ingredient': []
    }

    for igr in range(len(ing_groups)):
        group_name = ing_groups[igr].h2.text
        group_ings = ing_groups[igr].find_all('p', {'class': 'IngredientCard-name'})

        for i in range(len(group_ings)):
            all_ings['Group'].append(group_name)
            all_ings['Ingredient'].append(group_ings[i].text)
    
    all_ings_df = pd.DataFrame(all_ings)
        
    return all_ings_df

In [3]:
# Проверка: получение актуального списка ингредиентов
site_ings_df = get_ings(driver) 

In [4]:
site_ings_df

,Group,Ingredient
0,Бобовые,Горох сухой
1,Бобовые,Горошек зелёный
2,Бобовые,Маш (бобы мунг)
3,Бобовые,Нут (турецкий горох)
4,Бобовые,Фасоль стручковая (спаржевая)
...,...,...
220,Рыба и морепродукты,Хек
221,Рыба и морепродукты,Шпроты в масле
222,Рыба и морепродукты,Щука
223,Чай/Кофе,Какао


### **Вспомогательная функция для приведения символов строчки в нижний регистр и обрезания лишних пробелов по краям**

In [4]:
def str_lower(x):
    return str(x).strip().lower()

### **Функция для проверки наличия ингредиентов, подаваемых пользователем, в некотором перечне или на сайте**

**Аргументы**:
* driver - драйвер для Selenium
* my_ings - лист/ кортеж/ множество/ pd.Series с нужными ингредиентами, либо строчка с 1 ингредиентом (всё not case sensitive, т.е. можно вводить любыми буквами)
* site_ings - лист/ кортеж/ множество/ pd.Series с перечнем ингредиентов с сайта, либо словарь или pd.DataFrame с перечнем ингредиентов; для последних двух типов необходимо указать аргумент site_ings_ingkey
* site_ings_ingkey - ключ для словаря с ингредиентами с сайта или название колонки в pd.DataFrame с ингредиентами с сайта, соответствующие значениям/ колонке с перечнем ингредиентов. Пол умолчанию равен 'Ingredient'

**На выходе**:
* если все ингредиенты из my_ings есть в site_ings, напечатает и вернёт 'Все ингредиенты есть на сайте',
* в противном случае вернет None и для ингредиентов из my_ings, которые не будут найдены в site_ings, напечатает сообщение вида 'Ингредиента "N" нет на сайте' и далее перечень похожих ингредиентов, т.е. названий ингредентов из перечня/ с сайта, которые содержат те же слова, что и "не найденный" ингредиент, либо, если не найдется похожих, то сообщение 'Похожих ингредиентов не найдено'.

In [5]:
def check_ings(driver, my_ings, site_ings=None, site_ings_ingkey='Ingredient'):
    import pandas as pd
    
    # если перечень индикаторов с сайта не подан, запросить его
    if site_ings is None:
        site_ings = get_ings(driver)
     
    # проверим соответствие поданного перечня ингредиентов требованиям; 
    # в случае соответствия создадим множество уникальных ингредиентов и обрежем лишние пробелы по краям
    if isinstance(site_ings, str):
        site_ings_set = set([site_ings])
    elif isinstance(site_ings, (list, tuple, set, pd.Series)):
        site_ings_set = set(site_ings)
    elif isinstance(site_ings, (dict, pd.DataFrame)):
        if site_ings_ingkey not in site_ings.keys():
            print('Указанного ключа (аргумент site_ings_ingkey) нет в перечне ингредиентов с сайта (аргумент site_ings)')
            return
        else :
            site_ings_set = set(site_ings[site_ings_ingkey])
    else:
        print('Формат перечня ингредиентов с сайта (аргумент site_ings) не соответствует требованиям.')
        return
    
    site_ings_set = {str_lower(i) for i in site_ings_set}

    # проверим соответствие поданного перечня ингредиентов требованиям; 
    # в случае соответствия создадим множество уникальных ингредиентов и обрежем лишние пробелы по краям
    if isinstance(my_ings, str):
        my_ings_set = set([my_ings])
    elif isinstance(my_ings, (list, tuple, set, pd.Series)):
        my_ings_set = set(my_ings)
    else:
        print('Формат перечня ингредиентов не соответствует требованиям.')
        return

    my_ings_set = {str_lower(i) for i in my_ings_set}
    if len(my_ings_set) < 1 or len(my_ings_set) > 4:
        print('Количество запрашиваемых ингредиентов (аргумент my_ings) должно быть от 1 до 4.')
        return            

    # проверим, есть ли my_ings в site_ings
    import re
    ing_not_in_list = 0
    for ing in my_ings_set:
        if ing not in site_ings_set:
            ing_words = set(re.findall(r'\w+', ing))
            similar_ings = []
            for word in ing_words:
                similar_ings += [i for i in site_ings_set if i.find(word) != -1]
            print('Ингредиента "' + ing + '" нет на сайте :(')
            if len(similar_ings) == 0:
                print('Ингредиентов с похожими словами не нашлось :(')
            else:
                print('Похожие ингредиенты:')
                print(*set(similar_ings), sep=", ")
            ing_not_in_list += 1
    
    if ing_not_in_list == 0:
        print('Все ингредиенты есть на сайте :)')
        return 'Все ингредиенты есть на сайте'
    else:
        return

In [6]:
# Проверка 1
my_ings = ['горох сухой', 'горошек репчатый']
site_ings = site_ings_df
site_ings_ingkey='Ingredient'
check_ings(driver, my_ings, site_ings=site_ings_df)

Ингредиента "горошек репчатый" нет на сайте :(
Похожие ингредиенты:
лук репчатый, горошек зелёный


In [7]:
# Проверка 2
my_ings = ['горох сухой', 'картофель']
site_ings = site_ings_df
site_ings_ingkey='Ingredient'
check_ings(driver, my_ings, site_ings=site_ings_df)

Все ингредиенты есть на сайте :)


'Все ингредиенты есть на сайте'

In [8]:
# Проверка 3 (без подачи списка ингредиентов - запросит с сайта актуальный)
my_ings = ['горох сухой', 'картофель']
check_ings(driver, my_ings)

Все ингредиенты есть на сайте :)


'Все ингредиенты есть на сайте'

### **Функция для составления таблички с рецептами по списку ингредиентов**

**Аргументы**:
* driver - драйвер для Selenium
* my_ings - лист/ кортеж/ множество/ pd.Series с нужными ингредиентами, либо строчка с 1 ингредиентом (всё not case sensitive, т.е. можно вводить любыми буквами)

**На выходе**:
* если какого-либо ингредиента нет на сайте, напечатает перечень не найденных ингредиентов и перечень похожих по словам ингредиентов, вернёт None
* если все ингредиенты на сайте есть и если найден хотя бы один рецепт с ними, вернёт pd.DataFrame с тремя колонками: Название рецепта, Ингредиенты (кол-во), Ссылка на рецепт
* если все ингредиенты на сайте есть, но не найден ни один рецепт с ними, вернёт None и сообщение 'Ничего не найдено'

In [9]:
def find_recipes(driver, my_ings):
    from bs4 import BeautifulSoup
    from time import sleep
    import pandas as pd
    
    # проверим соответствие поданного перечня ингредиентов требованиям; 
    # в случае соответствия создадим множество уникальных ингредиентов и обрежем лишние пробелы по краям;
    # в противном случае вернём None и напечатаем предупреждение
    if isinstance(my_ings, str):
        my_ings_set = set([my_ings])
    elif isinstance(my_ings, (list, tuple, set, pd.Series)):
        my_ings_set = set(my_ings)
    else:
        print('Формат перечня ингредиентов не соответствует требованиям.')
        return

    my_ings_set = {str_lower(i) for i in my_ings_set}
    if len(my_ings_set) < 1 or len(my_ings_set) > 4:
        print('Количество запрашиваемых ингредиентов (аргумент my_ings) должно быть от 1 до 4.')
        return  
    
    # задан ли драйвер для Selenium?
    site_ings_df = get_ings(driver)
    if site_ings_df is None:
        return
    sleep(10)
 
    # проверим, все ли ингредиенты есть на сайте:
    # если нет, то вернёт None, а из функции check_ings напечатает предупреждения;
    # в противном случае продолжит выполнение
    ings_exist = check_ings(driver, my_ings, site_ings=site_ings_df)
    if ings_exist is None:
        driver.quit()
        return
    else:
        # столбец с названиями игредиентов строчными буквами для not case sensitive поиска
        site_ings_df['Ingredient_l'] = site_ings_df['Ingredient'].apply(str_lower)
        # список ингредиентов с названиями в том виде, в котором они присутствуют на сайте
        my_ings_list = list(site_ings_df['Ingredient'][site_ings_df['Ingredient_l'].isin(my_ings_set)])
        
        # выбираем ингредиенты на странице с перечнем ингредиентов
        for ing in my_ings_list:
            # находим картинку продукта по названию ингредиента (по ней тоже можно кликать для выбора)
            elem = driver.find_element_by_css_selector('img[alt="%s"]' % ing)

            # отскроллим страницу немножко, иначе верхняя панель с названием сайта и меню перекроет картинки с ингр-тами
            y = elem.location['y'] - 100
            driver.execute_script("window.scrollTo(0, %s)" % y)
            sleep(10)

            # кликнем по картинке с продуктом
            elem.click()
            sleep(10)

        # после выбора всех ингредиентов нажмём кнопку "Подобрать рецепты"
        driver.find_element_by_css_selector('button[type=submit]').click()
        sleep(10)

        # заберём страничку с найденными рецептами
        recs_page = driver.page_source
        
        recs_page = BeautifulSoup(recs_page)
        
        # найдём рецепты
        recs = recs_page.find_all('a', {'class': 'RecipeCard-name'})
        # если рецепты не помещаются на одной странице, то результаты выводятся в другом формате;
        # в этом случае будем забирать рецепты только с первой страницы (самые популярные)
        if len(recs) == 0:
            recs = recs_page.find_all('a', {'class': 'RecipeCardNew-headerLink'})
            # а если ничего не нашлось, то вернём None и напечатаем сообщение
            if len(recs) == 0:
                print('На сайте не нашлось рецептов с таким набором ингредиентов :(')
                driver.quit()
                return
            else:
                npages = 2
        else:
            npages = 1
            
        # создадим словарик со всеми найденными рецептами
        recs_dict = {
            'RecipeName': [],
            'Ingredients': [],
            'URL': []
        }
        
        # заполним названия и ссылки
        for rec in recs:
            recs_dict['RecipeName'].append(rec.text)
            recs_dict['URL'].append(rec.get('href'))
        
            # заполним список ингредиентов
            # если страниц более одной, то ингредиенты можно выцепить прямо со страницы со списком рецептов;
            # если страница одна, то придётся заходить по ссылке в каждый рецепт и брать список оттуда
            rec_ings_for_dict = []
            if npages == 2:
                rec_parent = rec.parent.parent.parent.parent
                rec_ings = rec_parent.find_all('li', {'class': 'RecipeCardNew-ingredientsOverlayListItem'})
                for rec_ing in rec_ings:
                    rec_ings_for_dict.append(rec_ing.text)
            else:
                driver.get(rec.get('href'))
                sleep(10)
                rec_ings_page = driver.page_source
                rec_ings_page = BeautifulSoup(rec_ings_page)
                rec_ings = rec_ings_page.find_all('a', {'class': 'RecipeIngredients-name'})
                for rec_ing in rec_ings:
                    try:
                        rec_ings_for_dict.append(rec_ing.text + ' — ' + rec_ing.find_next_sibling().text)
                    except:
                        rec_ings_for_dict.append(rec_ing.text)
                
            recs_dict['Ingredients'].append('; '.join(rec_ings_for_dict))
        
        # создадим pandas data frame
        recs_df = pd.DataFrame(recs_dict)
        
        driver.quit()
        print('Приятного аппетита!')
        return recs_df

#### Проверка 1: Нет ингредиента на сайте

In [10]:
from fake_useragent import UserAgent
from selenium import webdriver
    
#driver.quit()
    
useragent = UserAgent()
profile = webdriver.FirefoxProfile()
profile.set_preference('general.useragent.override', useragent.random)
driver = webdriver.Firefox(firefox_profile=profile)

In [11]:
my_ings = ['горох сухой', 'горошек репчатый']
find_recipes(driver, my_ings)

Ингредиента "горошек репчатый" нет на сайте :(
Похожие ингредиенты:
лук репчатый, горошек зелёный


#### Проверка 2: 1 страница с рецептами
(придётся подождать: будет заходить в каждый за ингредиентами...)

In [12]:
from fake_useragent import UserAgent
from selenium import webdriver
from time import sleep

#driver.quit()
    
useragent = UserAgent()
profile = webdriver.FirefoxProfile()
profile.set_preference('general.useragent.override', useragent.random)
driver = webdriver.Firefox(firefox_profile=profile)

In [13]:
my_ings = ['горох сухой', 'картофель']
my_recs = find_recipes(driver, my_ings)

Все ингредиенты есть на сайте :)
Приятного аппетита!


In [14]:
my_recs

,RecipeName,Ingredients,URL
0,Суп-пюре с горохом орегон,Аджика сухая — 15 г; Брынза — 100 г; Бульон ов...,https://webspoon.ru/receipt/sup-pyure-s-gorokh...
1,Суп гороховый постный в мультиварке,Аджика сухая — 0.25 ч. л.; Вода — 2 л; Горох с...,https://webspoon.ru/receipt/sup-gorokhovyjj-po...
2,Суп гороховый со свининой,Аджика сухая — 1 щепотка; Вода — 5 л; Горох су...,https://webspoon.ru/receipt/sup-gorokhovyjj-so...
3,Гороховый суп с курицей,Горох сухой колотый — 1 ст.; Картофель — 1 шт....,https://webspoon.ru/receipt/gorokhovyjj-sup-s-...
4,Гороховый суп-пюре с беконом и зелёным соусом,Бекон — 250 г; Бульон мясной — 2000 мл; Горох ...,https://webspoon.ru/receipt/gorokhovyjj-sup-py...
5,Суп гороховый с грибами,Белые грибы сушёные — 20 г; Вода — 2500 мл; Го...,https://webspoon.ru/receipt/sup-gorokhovyjj-s-...
6,Гороховый суп с тушёнкой,Вода — 3 л; Говядина тушёная консервы — 400 г;...,https://webspoon.ru/receipt/gorokhovyjj-sup-s-...
7,Постный гороховый суп,Горох сухой целый — 200 г; Картофель — 4 шт.; ...,https://webspoon.ru/receipt/postnyjj-gorokhovy...
8,Гороховый суп с клёцками в мультиварке,Вода — 2500 мл; Горох сухой целый — 100 г; Кар...,https://webspoon.ru/receipt/gorokhovyjj-sup-s-...
9,Гороховый суп в мультиварке,Аджика сухая — 0.25 ч. л.; Бульон куриный — 1 ...,https://webspoon.ru/receipt/gorokhovyjj-sup-v-...


In [15]:
# сохраним в Excel
my_recs.to_excel('Recipes_ГорохКартофель.xlsx', index=False)

#### Проверка 3: несколько страниц с рецептами
(тут будет быстрее, т.к. ингредиенты можно получить со страницы с рецептами)

In [19]:
from fake_useragent import UserAgent
from selenium import webdriver
from time import sleep

#driver.quit()
    
useragent = UserAgent()
profile = webdriver.FirefoxProfile()
profile.set_preference('general.useragent.override', useragent.random)
driver = webdriver.Firefox(firefox_profile=profile)

In [20]:
my_ings = 'рис'
my_recs = find_recipes(driver, my_ings)

Все ингредиенты есть на сайте :)
Приятного аппетита!


In [21]:
my_recs

,RecipeName,Ingredients,URL
0,Роллы Филадельфия,Авокадо — 1 шт.; Водоросли нори — 3 шт.; Лосос...,https://webspoon.ru/receipt/rolly-filadelfija
1,Рисовая мука,Рис круглозернистый — 2 ст.,https://webspoon.ru/receipt/risovaya-muka
2,Быстрый плов на сковородке с говядиной,Говядина мякоть — 300 г; Масло подсолнечное ра...,https://webspoon.ru/receipt/bystryj-plov-na-sk...
3,Закуска на Хэллоуин «Глазные яблоки»,Желатин — 1 ч. л.; Маслины без косточек — 3 шт...,https://webspoon.ru/receipt/zakuska-na-khehllo...
4,Рис с консервированным тунцом,Вода — 2 ст.; Масло сливочное — 2 ст. л.; Морк...,https://webspoon.ru/receipt/ris-s-konservirova...
5,Рисовый уксус в домашних условиях,Вода — 4 ст.; Дрожжи сухие — 0.25 ст. л.; Рис ...,https://webspoon.ru/receipt/risovyjj-uksus-v-d...
6,Роллы с огурцом,Вода — 350 мл; Водоросли нори — 4 шт.; Огурцы ...,https://webspoon.ru/receipt/rolly-s-ogurtsom
7,Ужин на неделю – 5 рецептов. Видео,Вода — 500 мл; Вода — 400 мл; Крупа гречневая ...,https://webspoon.ru/receipt/uzhin-na-nedelyu-v...
8,"Тыква, фаршированная рисом и курицей",Базилик сухой — 1 щепотка; Вода — 150 мл; Кури...,https://webspoon.ru/receipt/tykva-farshirovann...
9,Салат «Завтрак туриста»,Лук репчатый — 1000 г; Масло подсолнечное рафи...,https://webspoon.ru/receipt/zavtrak-turista


In [22]:
# сохраним в Excel
my_recs.to_excel('Recipes_Рис.xlsx', index=False)

#### Проверка 4: ингредиенты есть, нет рецептов

In [23]:
from fake_useragent import UserAgent
from selenium import webdriver
from time import sleep

#driver.quit()   

useragent = UserAgent()
profile = webdriver.FirefoxProfile()
profile.set_preference('general.useragent.override', useragent.random)
driver = webdriver.Firefox(firefox_profile=profile)

In [24]:
my_ings = {'Какао', 'Треска'}
my_recs = find_recipes(driver, my_ings)

Все ингредиенты есть на сайте :)
На сайте не нашлось рецептов с таким набором ингредиентов :(


#### Проверка 5: не задан драйвер для Selenium (не открыто новое окно в браузере)

In [25]:
my_ings = {'Какао', 'Треска'}
my_recs = find_recipes(driver, my_ings)

Задайте драйвер для Selenium (так, чтобы новое окно браузера было открыто).
